# Market Basket Algorithm (MBA)

In [1]:
## Import das bibliotecas
import pandas as pd
import numpy as np
import pandasql as ps
import os
import unidecode
from mlxtend.frequent_patterns import apriori, association_rules

# Opções do pandas
pd.options.display.max_rows    = 999
pd.options.display.max_columns = 999

In [2]:
## Definição defunções
def hot_encode(x):
        return 1 if x>=1 else 0

In [3]:
path = r'D:\PI3\MarketBasketAnalysis\Dataset'
dataset = pd.read_csv(path+r'\dataset.csv',sep=';')
clientes = pd.read_csv(path+r'\clientes.csv',sep=';')
grupo_cliente = pd.read_csv(path+r'\grupo_cliente.csv',sep=';')
itens = pd.read_csv(path+r'\itens.csv',sep=';')

In [4]:
dataset

,id_compra,cd_compra,id_item,id_cliente,id_grupo,dt_compra,nm_quantidade,nm_vl_item,nm_vl_total
0,0,489434,3423,0,0,2009-12-01,12,6.95,83.40
1,1,489434,2740,0,0,2009-12-01,12,6.75,81.00
2,2,489434,2742,0,0,2009-12-01,12,6.75,81.00
3,3,489434,1258,0,0,2009-12-01,48,2.10,100.80
4,4,489434,617,0,0,2009-12-01,24,1.25,30.00
...,...,...,...,...,...,...,...,...,...
417529,525456,538145,2207,4382,46,2010-12-09,12,2.95,35.40
417530,525457,538145,1478,4382,46,2010-12-09,6,2.55,15.30
417531,525458,538145,2780,4382,46,2010-12-09,6,2.95,17.70
417532,525459,538145,1656,4382,46,2010-12-09,12,1.65,19.80


In [5]:
clientes

,id_cliente,cd_cliente,cliente_nome,cliente_sexo
0,1136,12346,Mila,F
1,647,12347,Christy,F
2,613,12348,Leona,F
3,549,12349,Nina,F
4,631,12351,Dora,F
...,...,...,...,...
4378,1921,18283,Russel,M
4379,3738,18284,Ora,M
4380,1880,18285,Odessa,F
4381,1227,18286,Kaiden,M


In [6]:
grupo_cliente

,id_grupo,cd_grupo_cliente,cd_pais,tx_pais,cd_estado
0,0,0,GB,United Kingdom,"London, City of"
1,1,1,GB,United Kingdom,Birmingham
2,2,2,GB,United Kingdom,Manchester
3,3,3,GB,United Kingdom,Leeds
4,4,4,GB,United Kingdom,Newcastle upon Tyne
5,5,5,GB,United Kingdom,Leicester
6,6,6,GB,United Kingdom,Glasgow City
7,7,7,GB,United Kingdom,Liverpool
8,8,8,GB,United Kingdom,Portsmouth
9,9,9,GB,United Kingdom,Southampton


In [7]:
itens

,id_item,cd_item,tx_item
0,0,10002,INFLATABLE POLITICAL GLOBE
1,1,10080,GROOVY CACTUS INFLATABLE
2,2,10109,BENDY COLOUR PENCILS
3,3,10120,DOGGY RUBBER
4,4,10123C,HEARTS WRAPPING TAPE
...,...,...,...
4026,4026,PADS,PADS TO MATCH ALL CUSHIONS
4027,4027,POST,POSTAGE
4028,4028,SP1002,KID'S CHALKBOARD/EASEL
4029,4029,TEST001,This is a test product.


In [8]:
## Query das tabelas
query_tabelas = """
SELECT
    ds.id_compra,
    ds.cd_compra,
    it.cd_item,
    it.tx_item,
    cl.cd_cliente,
    cl.cliente_nome,
    gc.cd_grupo_cliente,
    gc.cd_pais,
    gc.tx_pais,
    gc.cd_estado,
    ds.dt_compra,
    ds.nm_quantidade,
    ds.nm_vl_item,
    ds.nm_vl_total	
FROM
    dataset ds
LEFT JOIN
    clientes cl
ON
    cl.id_cliente = ds.id_cliente
LEFT JOIN
    grupo_cliente gc
ON
    gc.id_grupo = ds.id_grupo
LEFT JOIN
    itens it
ON
    it.id_item = ds.id_item
"""
df = ps.sqldf(query_tabelas)
df

,id_compra,cd_compra,cd_item,tx_item,cd_cliente,cliente_nome,cd_grupo_cliente,cd_pais,tx_pais,cd_estado,dt_compra,nm_quantidade,nm_vl_item,nm_vl_total
0,0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,13085,James,0,GB,United Kingdom,"London, City of",2009-12-01,12,6.95,83.40
1,1,489434,79323P,PINK CHERRY LIGHTS,13085,James,0,GB,United Kingdom,"London, City of",2009-12-01,12,6.75,81.00
2,2,489434,79323W,WHITE CHERRY LIGHTS,13085,James,0,GB,United Kingdom,"London, City of",2009-12-01,12,6.75,81.00
3,3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",13085,James,0,GB,United Kingdom,"London, City of",2009-12-01,48,2.10,100.80
4,4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,13085,James,0,GB,United Kingdom,"London, City of",2009-12-01,24,1.25,30.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417529,525456,538145,35648,VINTAGE BEAD PINK PURSE,15211,Zariyah,49,GB,United Kingdom,Hertfordshire,2010-12-09,12,2.95,35.40
417530,525457,538145,22276,WASH BAG VINTAGE ROSE PAISLEY,15211,Zariyah,49,GB,United Kingdom,Hertfordshire,2010-12-09,6,2.55,15.30
417531,525458,538145,82494L,WOODEN FRAME ANTIQUE WHITE,15211,Zariyah,49,GB,United Kingdom,Hertfordshire,2010-12-09,6,2.95,17.70
417532,525459,538145,22469,HEART OF WICKER SMALL,15211,Zariyah,49,GB,United Kingdom,Hertfordshire,2010-12-09,12,1.65,19.80


In [9]:
## Tratamento dos dados
grupos_query = """
SELECT DISTINCT
    cd_grupo_cliente,
    cd_estado as tx_grupo_cliente
FROM
    df
ORDER BY
    1    
"""
grupos = ps.sqldf(grupos_query)
grupos['tx_grupo_cliente'] = grupos['tx_grupo_cliente'].replace(', City of','',regex=True).replace([" ","-"],"_",regex=True).str.lower()#.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
list_grupos = grupos['cd_grupo_cliente'].values
grupos

,cd_grupo_cliente,tx_grupo_cliente
0,0,london
1,1,birmingham
2,2,manchester
3,3,leeds
4,4,newcastle_upon_tyne
5,5,leicester
6,6,glasgow_city
7,7,liverpool
8,8,portsmouth
9,9,southampton


In [10]:
baskets = []
for i in list_grupos:
    bask = 'basket_' + grupos.loc[grupos['cd_grupo_cliente'] == i,'tx_grupo_cliente'].values[0]
    globals()[bask] = df[df['cd_grupo_cliente'] == i].groupby(['cd_cliente', 'tx_item'])['nm_quantidade'].sum().unstack().reset_index().fillna(0).set_index('cd_cliente').applymap(hot_encode)
    print(f" {i} - {bask}: ",globals()[bask].shape)
    baskets.append(bask)

 0 - basket_london:  (300, 3419)
 1 - basket_birmingham:  (47, 1278)
 2 - basket_manchester:  (6, 159)
 3 - basket_leeds:  (17, 479)
 4 - basket_newcastle_upon_tyne:  (15, 419)
 5 - basket_leicester:  (4, 1999)
 6 - basket_glasgow_city:  (68, 1457)
 7 - basket_liverpool:  (18, 499)
 8 - basket_portsmouth:  (6, 137)
 9 - basket_southampton:  (9, 285)
 10 - basket_nottingham:  (23, 880)
 11 - basket_bristol:  (2, 133)
 12 - basket_sheffield:  (25, 683)
 13 - basket_kingston_upon_hull:  (12, 544)
 14 - basket_edinburgh:  (11, 511)
 15 - basket_cardiff:  (7, 416)
 16 - basket_stoke_on_trent:  (4, 419)
 17 - basket_coventry:  (5, 219)
 18 - basket_reading:  (10, 363)
 19 - basket_belfast:  (16, 424)
 20 - basket_derby:  (4, 288)
 21 - basket_plymouth:  (8, 225)
 22 - basket_wolverhampton:  (14, 673)
 23 - basket_swansea:  (5, 219)
 24 - basket_milton_keynes:  (1, 30)
 25 - basket_aberdeen_city:  (1, 162)
 26 - basket_norfolk:  (1, 65)
 27 - basket_luton:  (1, 110)
 28 - basket_islington:  (

In [12]:
globals()[baskets[18]]

tx_item,10 COLOUR SPACEBOY PEN,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,3 HEARTS HANGING DECORATION RUSTIC,36 FOIL STAR CAKE CASES,36 PENCILS TUBE RED SPOTTY,36 PENCILS TUBE SKULLS,36 PENCILS TUBE WOODLAND,3D TRADITIONAL CHRISTMAS STICKERS,6 CROCHET STRAWBERRIES,6 RIBBONS RUSTIC CHARM,60 CAKE CASES DOLLY GIRL DESIGN,60 CAKE CASES VINTAGE CHRISTMAS,72 SWEETHEART FAIRY CAKE CASES,ABSTRACT CIRCLES NOTEBOOK,ADVENT CALENDAR GINGHAM SACK,ALPHABET STENCIL CRAFT,ANGEL DECORATION STARS ON DRESS,ASSORTED CAKES FRIDGE MAGNETS,ASSORTED CHEESE FRIDGE MAGNETS,ASSORTED COLOUR BIRD ORNAMENT,ASSORTED ICE CREAM FRIDGE MAGNETS,ASSORTED SANSKRIT MINI NOTEBOOK,ASSTD DESIGN RACING CAR PEN,BABY BOOM RIBBONS,BAKING SET 9 PIECE RETROSPOT,BAKING SET SPACEBOY DESIGN,BALLOON PUMP WITH 10 BALLOONS,BATHROOM SET LOVE HEART DESIGN,BIG DOUGHNUT FRIDGE MAGNETS,BIRDS MOBILE VINTAGE DESIGN,"BIRTHDAY CARD, RETRO SPOT",BISCUIT TIN VINTAGE GREEN,BISCUIT TIN VINTAGE RED,BLUE SPOTTY CUP,BLUE SPOTTY PLATE,BLUE 3 PIECE MINI DOTS CUTLERY SET,BLUE HARMONICA IN BOX,BLUE PADDED SOFT MOBILE,BLUE PARTY BAGS,BLUE PATCH PENCIL CASE PINK HEART,BLUE ROSE PATCH PURSE PINK BUTTERFL,BLUE SPOT CERAMIC DRAWER KNOB,BLUE SPOTS WRAP,BLUE SPOTTY BOWL,BLUE STRIPE CERAMIC DRAWER KNOB,BREAD BIN DINER STYLE IVORY,BREAD BIN DINER STYLE RED,BRIGHT BLUES RIBBONS,BROCADE RING PURSE,BROCANTE SHELF WITH HOOKS,BUNNY DECORATION MAGIC GARDEN,CARD BIRTHDAY COWBOY,CARD CAT AND TREE,CARD CHRISTMAS VILLAGE,CARD CIRCUS PARADE,CARD DOG AND BALL,CARD DOLLY GIRL,CARD HOLDER GINGHAM HEART,CARD WEDDING DAY,CARDHOLDER HOLLY WREATH METAL,CAROUSEL DESIGN WASHBAG,CERAMIC BOWL WITH LOVE HEART DESIGN,CERAMIC CAKE BOWL + HANGING CAKES,CERAMIC CAKE DESIGN SPOTTED MUG,CERAMIC CAKE DESIGN SPOTTED PLATE,CERAMIC CAKE STAND + HANGING CAKES,CERAMIC CHERRY CAKE MONEY BANK,CERAMIC PLATE LOVE HEART DESIGN,CERAMIC STRAWBERRY CAKE MONEY BANK,CERAMIC STRAWBERRY DESIGN MUG,CHARLOTTE BAG SUKI DESIGN,CHILDREN'S APRON DOLLY GIRL,CHILDRENS GARDEN GLOVES BLUE,CHILDRENS GARDEN GLOVES PINK,CHILDS APRON SPACEBOY DESIGN,CHILDS GARDEN BRUSH BLUE,CHILDS GARDEN BRUSH PINK,CHILDS GARDEN TROWEL BLUE,CHILDS GARDEN TROWEL PINK,CHOC TRUFFLE GOLD TRINKET POT,CHOCOLATE THIS WAY METAL SIGN,CHRISTMAS CARD SINGING ANGEL,CHRISTMAS GINGHAM HEART,CHRISTMAS LIGHTS 10 VINTAGE BAUBLES,CHRISTMAS RETROSPOT ANGEL WOOD,CHRISTMAS RETROSPOT TREE WOOD,CHRYSANTHEMUM NOTEBOOK,CIRCUS PARADE LUNCH BOX,COOK WITH WINE METAL SIGN,CREAM CUPID HEARTS COAT HANGER,CREAM SLICE FLANNEL PINK SPOT,"DECOUPAGE,GREETING CARD,",DENIM PATCH PURSE PINK BUTTERFLY,DINOSAUR KEYRINGS ASSORTED,DINOSAUR LUNCH BOX WITH CUTLERY,DINOSAUR PARTY BAG + STICKER SET,DOCTOR'S BAG SOFT TOY,DOLLY GIRL LUNCH BOX,DOOR MAT BLACK FLOCK,DOORMAT AIRMAIL,DOORMAT MULTICOLOUR STRIPE,EASTER TIN BUCKET,EASTER TIN BUNNY BOUQUET,EASTER TIN KEEPSAKE,ECONOMY LUGGAGE TAG,EDWARDIAN PARASOL BLACK,EDWARDIAN PARASOL NATURAL,EDWARDIAN PARASOL PINK,EDWARDIAN PARASOL RED,"ELEPHANT, BIRTHDAY CARD,",ENAMEL BLUE RIM BISCUIT BIN,"FANCY FONT BIRTHDAY CARD,",FANCY FONTS BIRTHDAY WRAP,FAWN AND MUSHROOM GREETING CARD,FAWN BLUE HOT WATER BOTTLE,FELT FARM ANIMAL WHITE BUNNY,FELT TOADSTOOL SMALL,FELT TOADSTOOL LARGE,FELTCRAFT CHRISTMAS FAIRY,FELTCRAFT DOLL EMILY,FELTCRAFT DOLL MARIA,FELTCRAFT DOLL MOLLY,FELTCRAFT DOLL ROSIE,FELTCRAFT PRINCESS CHARLOTTE DOLL,FELTCRAFT PRINCESS LOLA DOLL,FELTCRAFT PRINCESS OLIVIA DOLL,FIRST CLASS LUGGAGE TAG,FIVE CATS HANGING DECORATION,"FLOWER FAIRY,5 SUMMER B'DRAW LINERS",GARDEN PATH NOTEBOOK,GIN & TONIC DIET GREETING CARD,GIN + TONIC DIET METAL SIGN,GRAND CHOCOLATECANDLE,GREEN SPOTTY PLATE,GREEN 3 PIECE MINI DOTS CUTLERY SET,GREEN CHRISTMAS TREE CARD HOLDER,GREEN SPOTTY BOWL,GREEN SPOTTY CUP,GROW YOUR OWN PLANT IN A CAN,GUMBALL COAT RACK,HAND OVER THE CHOCOLATE SIGN,HAND WARMER UNION JACK,HEARTS GIFT TAPE,HEARTS WRAPPING TAPE,"HOOK, 1 HANGER ,MAGIC GARDEN","HOOK, 5 HANGER , MAGIC TOADSTOOL","HOOK, 5 HANGER ,MAGIC TOADSTOOL RED",HOT WATER BOTTLE BABU

In [12]:
frequencies = []
rules = []
for i in baskets:
    frq = 'frequent_' + i.replace('basket_','')
    rls = 'rules_' + i.replace('basket_','')
    print(f"Begin - {i.replace('basket_','')}")
    globals()[frq] = apriori(globals()[i],min_support=0.25,use_colnames=True)
    globals()[rls] = association_rules(globals()[frq],metric='lift',min_threshold=1).sort_values(['confidence', 'lift'], ascending =[False, False])
    print(f"End - {i.replace('basket_','')}")
    frequencies.append(frq)
    rules.append(rls)


Begin - london
End - london
Begin - birmingham
End - birmingham
Begin - manchester
End - manchester
Begin - leeds
End - leeds
Begin - newcastle_upon_tyne
End - newcastle_upon_tyne
Begin - leicester


MemoryError: cannot allocate array memory

In [ ]:
rules